<a href="https://colab.research.google.com/github/Ravali0726/mihir/blob/main/telecom_churn_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Overview:
* "The Orange Telecom's Churn Dataset, which consists of cleaned customer activity data (features), along with a churn label specifying whether a customer canceled the subscription, will be used to develop predictive models."

Objectives:
* Build a classification model to predict customer churn, which is a binary label
* Identify key drivers of customer churn and customer retention

Success criteria:
* Identify most of the customers that canceled the subscription (recall metric)

Link to data:
* https://www.kaggle.com/datasets/mnassrib/telecom-churn-datasets?select=churn-bigml-80.csv

### Imports

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from datetime import datetime
ts = datetime.now() # Notebook runtime

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_selection import chi2
from scipy.stats import ttest_ind
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler, TomekLinks

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier

### Load Data

In [ ]:
df = pd.read_csv('/kaggle/input/telecom-churn-datasets/churn-bigml-80.csv')
print(df.shape)
df.head()

In [ ]:
print(*df.columns, sep='\n')

### Understand Data

In [ ]:
def df_info():
    """Summary stats and other info about the data,
    like number of nulls, data types, unique values, and number of outliers"""

    # Continuous data outliers using IQR
    continuous_df = df.select_dtypes(include=[float, int])
    Q1 = continuous_df.quantile(0.25)
    Q3 = continuous_df.quantile(0.75)
    IQR = Q3 - Q1
    outliers = ((continuous_df < (Q1 - 1.5 * IQR)) | (continuous_df > (Q3 + 1.5 * IQR))).sum()
    outliers = outliers.to_frame(name='outliers')

    # Summary stats for continuous data
    descr = continuous_df.describe().T

    # Nulls, data types, and unique values
    nulls = df.isna().sum().to_frame(name='nulls')
    dtypes = df.dtypes.to_frame(name='dtype')
    nuniques = df.nunique().to_frame(name='nunique')
    info = pd.concat([nulls, dtypes, nuniques], axis=1)

    # Left join continuous data
    info = info.merge(outliers, left_index=True, right_index=True, how='left')
    info = info.merge(descr, left_index=True, right_index=True, how='left')
    return info

df_info()

### Prepare Data

In [ ]:
def prepare_data(df=df):
    # Shuffle the data bc idk how it was generated and if it's sorted
    df = (df
          .sample(frac=1)
          .reset_index(drop=True)
         )

    # Fix data types
    df['Churn'] = df['Churn'].astype('uint8') # boolean --> integer
    yesno_func = lambda x: 1 if x == 'Yes' else (0 if x == 'No' else None) # yes/no --> 1/0
    df['International plan'] = df['International plan'].apply(yesno_func).astype('uint8')
    df['Voice mail plan'] = df['Voice mail plan'].apply(yesno_func).astype('uint8')

    # One-hot encode 'State' and 'Area code'
    df = pd.get_dummies(df
                        , columns=['State', 'Area code']
                        , prefix={'State': 'State'
                                  , 'Area code': 'Area_code'
                                 }
                       )

    # Feature engineering: flag if the customer has both 'International plan' and 'Voice mail plan'
    df['intl_and_vm_plan'] = df['International plan'] * df['Voice mail plan']

    # Feature engineering: total and percent of total for 'calls', 'minutes', and 'charge' columns
    cols_dict = {'calls': [i for i in df.columns if 'calls' in i]
                 , 'minutes': [i for i in df.columns if 'minutes' in i]
                 , 'charge': [i for i in df.columns if 'charge' in i]
                }

    for k, v in cols_dict.items():
        sums = df[v].sum(axis=1)
        df[f'{k}_total'] = sums
        for col in v:
            df[f'pct_{col}'] = df[col] / sums

    return df

df = prepare_data()
print(df.shape)
df.head()

In [ ]:
# Define features and target
X = df.drop(columns=['Churn'])
y = df['Churn']
print(f'X: {X.shape}\ny: {y.shape}')

### Explore Data

In [ ]:
# Most customers are in West Virginia
# Very few customers are in California
(df[
    [i for i in df if 'State' in i]
]
 .sum()
 .sort_values()
 .plot(kind='barh'
       , figsize=(5,20)
      )
)

In [ ]:
# Check for 'Churn' class imbalance
pct_churn = (y
             .value_counts(normalize=True)
             .loc[1]
             * 100
            )
print(f'{pct_churn:.1f}% of customers churn')
y.value_counts()

In [ ]:
# Correlation matrix with all the features
plt.figure(figsize=(16,12))
sns.heatmap(X.corr()
            , cmap="RdBu_r"
            , vmin=-1
            , vmax=1)

In [ ]:
# Identify highly correlated features
# (Taken from page 70 on http://floodkingz.org/INZEPOCKET/machinelearningpocketreference.pdf)
def correlated_columns(df, threshold=0.95):
    return (
        df.corr()
        .pipe(
            lambda df1: pd.DataFrame(np.tril(df1, k=-1)
                                     , columns=df.columns
                                     , index=df.columns,
                                    )
        )
        .stack()
        .rename("correlation")
        .pipe(
            lambda s: s[s.abs() > threshold]
            .reset_index()
        ).query("level_0 not in level_1")
    )

colinears = correlated_columns(X)
colinears.sort_values('correlation', ascending=False)

In [ ]:
# Remove half of the highly correlated features
colinear_cols = colinears['level_0'].tolist()
X.drop(columns=colinear_cols, inplace=True)
print(f'X: {X.shape}')

In [ ]:
# Correlation between the features and the target
# (Most features have zero correlation with the target)
with plt.style.context('ggplot'):
    plt.figure(figsize=(10,6))
    (df
     .corr()
     .loc[X.columns, y.name]
     .sort_values(ascending=False)
     .hist(alpha=0.3)
    )
    plt.xlabel("Correlation with 'Churn'")
    plt.ylabel('Frequency')
    plt.title('Distribution of the correlation between the features and the target\n(Most features have zero correlation with the target)')

In [ ]:
# Explore categorical features
# Plot the crosstab heatmap for each of the categorical features vs churn/non-churn
# Use the chi-squared test to check whether there's a significant difference between the observed and expected frequencies
X_cat = X.select_dtypes(exclude=[float, int])
chi_t, chi_p = chi2(X_cat, y)
chi_res = pd.Series(chi_p
                    , index=X_cat.columns
                    , name='Chi2_Pvalues')

# Print most significant from the chi-squared test
n_cat_imp = 5
cat_imp = chi_res.sort_values().iloc[:n_cat_imp].index.tolist()
print(f'In descending order, the {n_cat_imp} biggest differences between observed and expected values are:', *cat_imp, sep='\n')

# Plot
n_rows = 12
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 40))
fig.suptitle('Heat Maps of the Crosstabs between each Categorical Feature and Churn\nAnd the Chi-squared P-values\n(X-axis is Churn and Y-axis is the Categorical Feature)')
for i, ax in zip(range(X_cat.shape[1]), axes.flat):
    to_plot = X_cat.iloc[:,i]
    a = pd.crosstab(to_plot, y)
    aa = a / a.sum().sum() # Counts --> percent of total (bc it's easier to understand)
    sns.heatmap(aa
                , cmap="RdBu_r"
                , annot=True
                , fmt='.2f'
                , vmin=0
                , vmax=1
                , ax=ax)
    ax.set_title(f'{to_plot.name}: {chi_res.loc[to_plot.name]:.3f}')
    ax.set_xlabel('')
    ax.set_ylabel('')

In [ ]:
# Explore continuous features
# Plot the distributions of the continuous features
# Segment the distributions by churn/non-churn
# Balance the target classes so it's easier to understand the distribution of the feature for each of the classes
# Use a t-test to measure whether there's a significant difference between the means (churn vs non-churn)
rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X, y)
X_cont = X_res.select_dtypes(include=[float, int])

rus_df = pd.concat([X_cont, y_res], axis=1)
x1 = rus_df[rus_df['Churn'] == 1].drop(columns=['Churn'])
x2 = rus_df[rus_df['Churn'] == 0].drop(columns=['Churn'])
ttest_stat, ttest_pval = ttest_ind(x1, x2)
ttest_res = pd.Series(ttest_pval
                      , name='Ttest_Pvalues'
                      , index=x1.columns)

# Print most significant from the t-test
n_cont_imp = 5
cont_imp = ttest_res.sort_values().iloc[:n_cont_imp].index.tolist()
print(f'In descending order, the {n_cont_imp} biggest differences between the churn/non-churn means are:', *cont_imp, sep='\n')

# Plot
n_rows = 5
n_cols = 5
fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 18))
fig.suptitle('Distributions of each Continuous Feature and Segmented by Churn\nAnd the T-test P-values')
for i, ax in zip(range(X_cont.shape[1]), axes.flat):
    to_plot = X_cont.iloc[:,i]
    sns.histplot(x=to_plot
                 , hue=y_res
                 , element='poly'
                 , ax=ax)
    ax.set_title(f'{to_plot.name}: {ttest_res.loc[to_plot.name]:.3f}')
    ax.set_xlabel('')
    ax.set_ylabel('')

In [ ]:
def scaler_func(data):
    """Scale the continuous features and add back the discrete features"""
    cont_df = data.select_dtypes(include=[float, int])
    dis_df = data.select_dtypes(exclude=[float, int])
    scaler = StandardScaler()
    sca = scaler.fit_transform(cont_df)
    sca_df = pd.DataFrame(sca, columns=cont_df.columns, index=cont_df.index)
    scaled_df = pd.concat([sca_df, dis_df], axis=1)
    return scaled_df

In [ ]:
# Parallel coordinates plotting to further understand how the continuous features relate to 'Churn'
plt.figure(figsize=(20,5))
ax = pd.plotting.parallel_coordinates(frame=scaler_func(df)
                                      , class_column=y.name
                                      , cols=df.select_dtypes(include=[float, int]).columns
                                      , color=('#FF5733', '#6EFF33')
                                     )
plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')
plt.xlabel('Continuous Features')
plt.ylabel('Scaled Value')
plt.title('Parallel Coordinates between each of the Continous Features and Churn')
plt.show()

In [ ]:
# Use all the features (categorical & continuous),
# to fit a decision tree to understand which features are most useful in separating churn/non-churn customers
tree = DecisionTreeClassifier()
tree.fit(X, y)
tree_imp = pd.Series(tree.feature_importances_
                    , name='tree_imp'
                    , index=X.columns)

# Plot feature importance
# The importance measures the average gain of purity by splits of a given variable (Gini importance)
(tree_imp
 .sort_values()
 .plot(kind='barh'
       , figsize=(5,20)
       , title='Decision Tree Feature Importance'
       , xlabel='Feature'
       , ylabel='Importance')
)

In [ ]:
# Fit and plot a decision tree with the most important features to build intuition about customers that churn vs don't churn
n_most_imp = 6
most_imp = (tree_imp
            .sort_values()
            .iloc[-n_most_imp:]
            .index
            .tolist()
           )

tree_2 = DecisionTreeClassifier(max_depth=5) # Limit the depth so it's easier to understand
tree_2.fit(X[most_imp], y)

# Plot
plt.figure(figsize=(40,20))
plot_tree(tree_2
          , feature_names=most_imp
          , class_names=[str(i) for i in sorted(y.unique())]
          , filled=True
          , fontsize=16
         )
plt.title(f'Decision tree with the {n_most_imp} most important features', fontsize=20)
plt.show()

### Sample Data

In [ ]:
# Partition training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y
                                                    , test_size=0.3
                                                    , stratify=y
                                                    , random_state=42
                                                   )
# Training data
n_tr = len(X_train)
pct_tr = (y_train
          .value_counts(normalize=True)
          .loc[1]
          *100
         )
vc_tr = y_train.value_counts()
print(f' -- Training data -- \nSample size: {n_tr:,.0f}\nPositive class: {pct_tr:,.1f}%\n{vc_tr}\n')


# Testing data
n_te = len(X_test)
pct_te = (y_test
          .value_counts(normalize=True)
          .loc[1]
          *100
         )
vc_te = y_test.value_counts()
print(f' -- Testing data -- \nSample size: {n_te:,.0f}\nPositive class: {pct_te:,.1f}%\n{vc_te}')

### Model Selection

In [ ]:
# Model selection process:
# Only use the training data from the train_test_split
# Cross validate for a more robust assessment of model performance
# Scale the cross validation training and testing data separately to prevent data leakage
# Balance the target class in the cross validation training data in two steps:
    # 1) Over-sample the minority class ('Churn'=1) with SMOTE-NC, which handles categorical and continuous features
    # 2) Under-sample the orignal majority class ('Churn'=0) by removing Tomek links to increase the separation between classes
# Fit the model with the cross validation training data
# Score the model with the cross validation testing data
# Measure the prediction error
# (I'm mostly interested in the recall score, though I also included a few other metrics for overall performance)

# Classification models
models = [KNeighborsClassifier()
          , LogisticRegression(random_state=123, solver='liblinear') # The default solver could not converge
          , GaussianNB()
          , RandomForestClassifier(random_state=123)
          , LGBMClassifier(random_state=123)
         ]

X_cat_mask = [True if X[i].dtype == 'uint8' else False for i in X.columns] # Mask categorical features for the resampling
n_folds = 8
kf = KFold(n_splits=n_folds)
res = {}
for i, (train_index, test_index) in enumerate(kf.split(X_train)):
    # Training data
    X_train_fold = X_train.iloc[train_index]
    y_train_fold = y_train.iloc[train_index]

    # Testing data
    X_test_fold = X_train.iloc[test_index]
    y_test_fold = y_train.iloc[test_index]

    # Scale the training and testing data separately
    X_train_fold = scaler_func(X_train_fold)
    X_test_fold = scaler_func(X_test_fold)

    # Over-sample the minority class ('Churn'=1) in the training data
    sm = SMOTENC(random_state=42, categorical_features=X_cat_mask) # Specify which features are categorical
    X_train_fold, y_train_fold = sm.fit_resample(X_train_fold, y_train_fold)

    # Under-sample the 'original' majority class ('Churn'=0) in the over-sampled training data by removing Tomek links
    # (Tomek links are pairs of instances of opposite classes who are their own nearest neighbors)
    tl = TomekLinks(sampling_strategy=[0])
    X_train_fold, y_train_fold = tl.fit_resample(X_train_fold, y_train_fold)

    for model in models:
        # Fit the model with the training data
        model.fit(X_train_fold, y_train_fold)

        # Score the model with the testing data
        y_pred = model.predict(X_test_fold)
        y_pred_proba = model.predict_proba(X_test_fold)

        # Measure the prediction error
        recall = metrics.recall_score(y_test_fold, y_pred)
        f1 = metrics.f1_score(y_test_fold, y_pred)
        auc = metrics.roc_auc_score(y_test_fold, y_pred)
        logloss = metrics.log_loss(y_test_fold, y_pred_proba)

        # Record results
        name = model.__class__.__name__
        res[f'{name}_{i}'] = {'Model': name
                              , 'Iter': i
                              , 'Recall': recall
                              , 'F1': f1
                              , 'AUC': auc
                              , 'Logloss': logloss
                             }

# Plot results
res = pd.DataFrame.from_dict(res).T.reset_index(drop=True)
(res
 .groupby('Model')
 .mean()
 .drop(columns=['Iter'])
 .style
 .highlight_max(color='lightgreen', subset=['Recall', 'F1', 'AUC']) # higher is better --> highlight_max
 .highlight_min(color='lightgreen', subset=['Logloss']) # lower is better --> highlight_min
 .format(precision=2)
)

### Feature Importance

In [ ]:
# Fit the best model on all the data

# Scale data
X_ = scaler_func(X)

# Over-sample the minority class ('Churn'=1)
X_cat_mask = [True if X[i].dtype == 'uint8' else False for i in X.columns]
sm = SMOTENC(random_state=42, categorical_features=X_cat_mask)
X_, y_ = sm.fit_resample(X_, y)

# Under-sample the 'original' majority class ('Churn'=0)
tl = TomekLinks(sampling_strategy=[0])
X_, y_ = tl.fit_resample(X_, y_)

# Fit model
clf = LGBMClassifier(random_state=123)
clf.fit(X_, y_)

# Feature importance
feature_importances = pd.Series(clf.feature_importances_
                               , name='imp'
                               , index=X.columns)

# Plot
(feature_importances
 .sort_values()
 .plot(kind='barh', figsize=(8,24))
)

### Submission

In [ ]:
# Load the smaller dataset for the final prediction
df20 = pd.read_csv('/kaggle/input/telecom-churn-datasets/churn-bigml-20.csv')

# Prepare data
df20 = prepare_data(df=df20)

# Features and target
X20 = df20[X.columns]
y20 = df20[y.name]

# Scale features
X20 = scaler_func(X20)

# Prediction
submission = clf.predict(X20)

# Recall
submission_recall = metrics.recall_score(y20, submission)
print(f'Final submission recall: {submission_recall:.2f}')

In [ ]:
# Notebook runtime
print(datetime.now() - ts)